In [157]:
import pandas as pd
import numpy as np
import random
import pickle


In [158]:
data_orig= pd.read_csv('../MA_PREDICTOR/data/ma_data_car.csv', parse_dates=['announcement_date'])

In [159]:
data = data_orig[(data_orig.car > -0.3) & (data_orig.car < 0.3)]

In [160]:
data.head()

,id,announcement_date,deal_value,acquiror_total_assets,acquiror_name,acquiror_ticker,target_name,target_nation,acquiror_nation,target_status,...,shares_acquired,consideration_offered,attitude,acquisition_technique,acquiror_financial_advisor,target_financial_advisor,purpose,acquiror_code,target_code,car
0,1631732020,2005-01-01,12.00,3160.41,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,100.0,Cash|Cash Only,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,50102030,50103030,0.006854
1,1653652020,2005-01-01,NaN,1680.57,Regis Corp,RGS,Scot Lewis Schools,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,54201030,63103010,-0.010266
2,1949421020,2005-01-01,NaN,3301.76,Google Inc,GOOG,PhatBits,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,57201030,57201020,0.007746
3,1652453040,2005-01-01,20.03,371.71,Graco Inc,GGG,Gusmer Europe SL,Spain,United States,Private,...,100.0,Cash|Cash Only,Friendly,Not Applicable,NaN,NaN,NaN,52102010,51101010,-0.011133
4,2721963020,2005-01-01,18220.52,NaN,Exxon Mobil Corp,XOM,Exxon Mobil Corp,United States,United States,Public,...,5.0,Cash|Cash Only,No Applicable,Open Market Purchase|Privately Negotiated Purc...,NaN,NaN,Other,50102030,50102030,-0.003971


In [161]:
data.columns

Index(['id', 'announcement_date', 'deal_value', 'acquiror_total_assets',
       'acquiror_name', 'acquiror_ticker', 'target_name', 'target_nation',
       'acquiror_nation', 'target_status', 'acquiror_industry',
       'target_industry', 'shares_at_announcement', 'shares_acquired',
       'consideration_offered', 'attitude', 'acquisition_technique',
       'acquiror_financial_advisor', 'target_financial_advisor', 'purpose',
       'acquiror_code', 'target_code', 'car'],
      dtype='object')

In [162]:
tmp = data[['consideration_offered', 'car', 'shares_acquired', 'shares_at_announcement','acquiror_code', 'target_code', 'acquiror_nation', 'target_nation', 'announcement_date']].copy()

# Consideration

In [163]:
# transform pipes into list of all considerations
list_gen = lambda x: x.split('|')
tmp['consideration_offered'] = tmp.consideration_offered.apply(list_gen)

# Cluster considerations
def cash(considerations):
    
    for consideration in considerations:
        if 'Cash' not in consideration:
            return 'Other'
    return 'Cash'

# Apply
tmp['consideration_offered'] = tmp.apply(lambda row: cash(row.consideration_offered), axis=1)

# Month

In [164]:
tmp['month']= pd.DatetimeIndex(tmp['announcement_date']).month

In [165]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,month
0,Cash,0.006854,100.0,0.0,50102030,50103030,United States,United States,2005-01-01,1
1,Other,-0.010266,100.0,0.0,54201030,63103010,United States,United States,2005-01-01,1
2,Other,0.007746,100.0,0.0,57201030,57201020,United States,United States,2005-01-01,1
3,Cash,-0.011133,100.0,0.0,52102010,51101010,United States,Spain,2005-01-01,1
4,Cash,-0.003971,5.0,0.0,50102030,50102030,United States,United States,2005-01-01,1
...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,100.0,0.0,52102010,59103010,United States,United States,2021-08-30,8
18455,Other,-0.040156,100.0,0.0,53203020,53205020,United States,United States,2021-08-30,8
18456,Other,-0.003545,100.0,0.0,54301020,57201010,United States,Norway,2021-08-30,8
18457,Other,-0.025992,100.0,0.0,55101010,52203030,United States,United States,2021-08-31,8


# Full vs not_full acqu

In [166]:
tmp['shares_acquired']=tmp['shares_acquired'].astype(str)

In [167]:
# Apply
tmp['shares_acquired']=tmp['shares_acquired'].apply(lambda val: "full" if val=="100.0" else "not_full")


In [168]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,month
0,Cash,0.006854,full,0.0,50102030,50103030,United States,United States,2005-01-01,1
1,Other,-0.010266,full,0.0,54201030,63103010,United States,United States,2005-01-01,1
2,Other,0.007746,full,0.0,57201030,57201020,United States,United States,2005-01-01,1
3,Cash,-0.011133,full,0.0,52102010,51101010,United States,Spain,2005-01-01,1
4,Cash,-0.003971,not_full,0.0,50102030,50102030,United States,United States,2005-01-01,1
...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,0.0,52102010,59103010,United States,United States,2021-08-30,8
18455,Other,-0.040156,full,0.0,53203020,53205020,United States,United States,2021-08-30,8
18456,Other,-0.003545,full,0.0,54301020,57201010,United States,Norway,2021-08-30,8
18457,Other,-0.025992,full,0.0,55101010,52203030,United States,United States,2021-08-31,8


# Shares at annoncement

In [169]:
tmp['shares_at_announcement']=tmp['shares_at_announcement'].astype(str)

In [170]:
# Apply
tmp['shares_at_announcement']=tmp['shares_at_announcement'].apply(lambda val: "no" if val=="0.0" else "yes")


In [171]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,month
0,Cash,0.006854,full,no,50102030,50103030,United States,United States,2005-01-01,1
1,Other,-0.010266,full,no,54201030,63103010,United States,United States,2005-01-01,1
2,Other,0.007746,full,no,57201030,57201020,United States,United States,2005-01-01,1
3,Cash,-0.011133,full,no,52102010,51101010,United States,Spain,2005-01-01,1
4,Cash,-0.003971,not_full,no,50102030,50102030,United States,United States,2005-01-01,1
...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,United States,United States,2021-08-30,8
18455,Other,-0.040156,full,no,53203020,53205020,United States,United States,2021-08-30,8
18456,Other,-0.003545,full,no,54301020,57201010,United States,Norway,2021-08-30,8
18457,Other,-0.025992,full,no,55101010,52203030,United States,United States,2021-08-31,8


# Cross-border vs National

In [172]:
tmp['cross_border'] = tmp.apply(lambda row: 'cross_border' if row.acquiror_nation == row.target_nation else 'national', axis=1)

In [173]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,month,cross_border
0,Cash,0.006854,full,no,50102030,50103030,United States,United States,2005-01-01,1,cross_border
1,Other,-0.010266,full,no,54201030,63103010,United States,United States,2005-01-01,1,cross_border
2,Other,0.007746,full,no,57201030,57201020,United States,United States,2005-01-01,1,cross_border
3,Cash,-0.011133,full,no,52102010,51101010,United States,Spain,2005-01-01,1,national
4,Cash,-0.003971,not_full,no,50102030,50102030,United States,United States,2005-01-01,1,cross_border
...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,United States,United States,2021-08-30,8,cross_border
18455,Other,-0.040156,full,no,53203020,53205020,United States,United States,2021-08-30,8,cross_border
18456,Other,-0.003545,full,no,54301020,57201010,United States,Norway,2021-08-30,8,national
18457,Other,-0.025992,full,no,55101010,52203030,United States,United States,2021-08-31,8,cross_border


# Industry relatedness

In [174]:
# Transform codes

def relatedness(acquiror, target):
    if acquiror == target:
        return 'industry'
    elif acquiror[:6] == target[:6]:
        return 'industry_group'
    elif acquiror[:4] == target[:4]:
        return 'business_sector'
    elif acquiror[:2] == target[:2]:
        return 'economic_sector'
    else:
        return 'not_related'

In [175]:
tmp['relatedness'] = data.apply(lambda row: relatedness(str(row.acquiror_code), str(row.target_code)), axis=1)

In [176]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,acquiror_nation,target_nation,announcement_date,month,cross_border,relatedness
0,Cash,0.006854,full,no,50102030,50103030,United States,United States,2005-01-01,1,cross_border,business_sector
1,Other,-0.010266,full,no,54201030,63103010,United States,United States,2005-01-01,1,cross_border,not_related
2,Other,0.007746,full,no,57201030,57201020,United States,United States,2005-01-01,1,cross_border,industry_group
3,Cash,-0.011133,full,no,52102010,51101010,United States,Spain,2005-01-01,1,national,not_related
4,Cash,-0.003971,not_full,no,50102030,50102030,United States,United States,2005-01-01,1,cross_border,industry
...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,United States,United States,2021-08-30,8,cross_border,not_related
18455,Other,-0.040156,full,no,53203020,53205020,United States,United States,2021-08-30,8,cross_border,business_sector
18456,Other,-0.003545,full,no,54301020,57201010,United States,Norway,2021-08-30,8,national,not_related
18457,Other,-0.025992,full,no,55101010,52203030,United States,United States,2021-08-31,8,cross_border,not_related


# Declassifier

In [177]:
# transform code into different cols mentioned above
def declassifier(x):
    x = str(x)
    return x[:2], x[:4], x[:6], x

In [178]:
tmp['economic_sector'], tmp['business_sector'], tmp['industry_group'], tmp['industry'] = \
zip(*data['acquiror_code'].map(declassifier))

# Columns modification

In [179]:
tmp.drop(['acquiror_nation', 'target_nation', 'announcement_date'], axis=1, inplace=True)

In [180]:
tmp

,consideration_offered,car,shares_acquired,shares_at_announcement,acquiror_code,target_code,month,cross_border,relatedness,economic_sector,business_sector,industry_group,industry
0,Cash,0.006854,full,no,50102030,50103030,1,cross_border,business_sector,50,5010,501020,50102030
1,Other,-0.010266,full,no,54201030,63103010,1,cross_border,not_related,54,5420,542010,54201030
2,Other,0.007746,full,no,57201030,57201020,1,cross_border,industry_group,57,5720,572010,57201030
3,Cash,-0.011133,full,no,52102010,51101010,1,national,not_related,52,5210,521020,52102010
4,Cash,-0.003971,not_full,no,50102030,50102030,1,cross_border,industry,50,5010,501020,50102030
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,0.020108,full,no,52102010,59103010,8,cross_border,not_related,52,5210,521020,52102010
18455,Other,-0.040156,full,no,53203020,53205020,8,cross_border,business_sector,53,5320,532030,53203020
18456,Other,-0.003545,full,no,54301020,57201010,8,national,not_related,54,5430,543010,54301020
18457,Other,-0.025992,full,no,55101010,52203030,8,cross_border,not_related,55,5510,551010,55101010


In [181]:
tmp= tmp.iloc[:, [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12, 1]]

In [182]:
tmp

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,month,cross_border,relatedness,economic_sector,business_sector,industry_group,industry,car
0,Cash,full,no,50102030,50103030,1,cross_border,business_sector,50,5010,501020,50102030,0.006854
1,Other,full,no,54201030,63103010,1,cross_border,not_related,54,5420,542010,54201030,-0.010266
2,Other,full,no,57201030,57201020,1,cross_border,industry_group,57,5720,572010,57201030,0.007746
3,Cash,full,no,52102010,51101010,1,national,not_related,52,5210,521020,52102010,-0.011133
4,Cash,not_full,no,50102030,50102030,1,cross_border,industry,50,5010,501020,50102030,-0.003971
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,Cash,full,no,52102010,59103010,8,cross_border,not_related,52,5210,521020,52102010,0.020108
18455,Other,full,no,53203020,53205020,8,cross_border,business_sector,53,5320,532030,53203020,-0.040156
18456,Other,full,no,54301020,57201010,8,national,not_related,54,5430,543010,54301020,-0.003545
18457,Other,full,no,55101010,52203030,8,cross_border,not_related,55,5510,551010,55101010,-0.025992


# Implementation of the declassifier function

In [183]:
pickle_in = open("/Users/manonlaffly/code/cobergmann/MA_PREDICTOR/MA_PREDICTOR/data/declassification.pkl","rb")
results = pickle.load(pickle_in)

In [184]:
results.head()

,economic_sector,business_sector,industry_group,industry,activity,hierarchical_id
0,Energy,Energy - Fossil Fuels,Coal,Coal,Coal (NEC),50101010
1,Energy,Energy - Fossil Fuels,Coal,Coal,Coal Mining Support,50101010
2,Energy,Energy - Fossil Fuels,Coal,Coal,Coal Wholesale,50101010
3,Energy,Energy - Fossil Fuels,Oil & Gas,Integrated Oil & Gas,Integrated Oil & Gas,50102010
4,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Exploration and Production,Oil & Gas Exploration and Production (NEC),50102020


In [185]:
def get_info_trbc(hierarchical_id):
    str_id= str(hierarchical_id)
    try:
        if len(str_id)==2:
            s = results.loc[results['hierarchical_id'].str.startswith(str_id), 'economic_sector'].reset_index(drop=True)
            return s[0]
        elif len(str_id)==4:
            r = results.loc[results['hierarchical_id'].str.startswith(str_id), 'business_sector'].reset_index(drop=True)
            return r[0]
        elif len(str_id)==6:
            q = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry_group'].reset_index(drop=True)
            return q[0] 
        elif len(str_id)==8:
            p = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry'].reset_index(drop=True)
            return p[0] 
        else:
            print("Please enter a valid hierarchical ID.")
    except:
        print("Please enter a valid hierarchical ID.")

In [186]:
tmp['economic_sector'] = tmp.apply(lambda row: get_info_trbc(row.economic_sector), axis=1)

In [187]:
tmp['business_sector'] = tmp.apply(lambda row: get_info_trbc(row.business_sector), axis=1)

In [188]:
tmp['industry_group'] = tmp.apply(lambda row: get_info_trbc(row.industry_group), axis=1)

In [189]:
tmp['industry'] = tmp.apply(lambda row: get_info_trbc(row.industry), axis=1)

In [190]:
tmp.head()

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,month,cross_border,relatedness,economic_sector,business_sector,industry_group,industry,car
0,Cash,full,no,50102030,50103030,1,cross_border,business_sector,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Refining and Marketing,0.006854
1,Other,full,no,54201030,63103010,1,cross_border,not_related,Consumer Non-Cyclicals,Personal & Household Products & Services,Personal & Household Products & Services,Personal Services,-0.010266
2,Other,full,no,57201030,57201020,1,cross_border,industry_group,Technology,Software & IT Services,Software & IT Services,Online Services,0.007746
3,Cash,full,no,52102010,51101010,1,national,not_related,Industrials,Industrial Goods,"Machinery, Tools, Heavy Vehicles, Trains & Ships",Industrial Machinery & Equipment,-0.011133
4,Cash,not_full,no,50102030,50102030,1,cross_border,industry,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Refining and Marketing,-0.003971


# Uploading

In [191]:
tmp.to_csv('../MA_PREDICTOR/data/ma_data_car_clean.csv', index=False, date_format="%d/%m/%Y")